# Aplicando CNN para classificação de fissura em concreto armado e pavimento

Nestes notebooks será aplicado redes pré-treinadas para detecção de fissura, neste dataset de imagens, o foco é comparar a redes pré-treinadas e ver qual seria a rede com o melhor desempenho

In [1]:
#importando as blibiotecas

from tensorflow import keras
from keras.layers import Dense, Flatten
from tensorflow.keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16, ResNet50,DenseNet121, MobileNet

import tensorflow as tf

In [2]:
# importando a blibioteca split-folders para dividir as imagens em treino e teste
!pip install split-folders

import splitfolders
splitfolders.ratio('/kaggle/input/concrete-and-pavement-crack-images', 
                   output="output", seed=1337, ratio=(.7, 0.3))

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Copying files: 30000 files [01:20, 374.41 files/s] 


In [3]:
BATCH = 26
IMG_WIDTH = 96
IMG_HEIGHT = 96

In [4]:
train_loader = tf.keras.preprocessing.image_dataset_from_directory(
"./output/train",
seed=123,
image_size=(IMG_HEIGHT, IMG_WIDTH),
batch_size=BATCH,label_mode= 'categorical'
)

test_loader = tf.keras.preprocessing.image_dataset_from_directory(
"./output/val",
seed=123,
image_size=(IMG_HEIGHT, IMG_WIDTH),
batch_size=BATCH, label_mode= 'categorical')

Found 21000 files belonging to 2 classes.
Found 9000 files belonging to 2 classes.


## Criando o modelo MobileNet

In [5]:
model = MobileNet(input_shape=(96, 96, 3), weights='imagenet', include_top=False,
             pooling='max',classifier_activation='softmax')
model.layers[0].trainable = False
model = Model(inputs=model.input, outputs=model.get_layer(index=-2).output)

/tmp/ipykernel_1411/13614470.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model = MobileNet(input_shape=(96, 96, 3), weights='imagenet', include_top=False,


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [6]:
#Colocando as duas últimas camadas
newTop = Sequential()
newTop.add(Flatten(input_shape=model.output_shape[1:]))
newTop.add(Dense(2, activation='softmax'))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
metricas = ['accuracy',
            keras.metrics.Precision(name='Pre'),keras.metrics.Recall(name='Rec')] 
#métricas que vão ser utilizadas para teste

In [8]:
model = Model(inputs=model.input, outputs=newTop(model.output))
model.summary()

optimizer = 'adam'
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=metricas)

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 48, 48, 32)     │           864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_bn (BatchNormalization)   │ (None, 48, 48, 32)     │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 48, 48, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1 (DepthwiseConv2D)     │ (None, 48, 48, 32)     │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_bn                    │ (None, 48, 48, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 48, 48, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1 (Conv2D)              │ (None, 48, 48, 64)     │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_bn                    │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pad_2 (ZeroPadding2D)      │ (None, 49, 49, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2 (DepthwiseConv2D)     │ (None, 24, 24, 64)     │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_bn                    │ (None, 24, 24, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2 (Conv2D)              │ (None, 24, 24, 128)    │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_bn                    │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 24, 24, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3 (DepthwiseConv2D)     │ (None, 24, 24, 128)    │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_bn                    │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 24, 24, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3 (Conv2D)              │ (None, 24, 24, 128)    │        16,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3_bn                    │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 3,247,298 (12.39 MB)

 Trainable params: 3,225,410 (12.30 MB)

 Non-trainable params: 21,888 (85.50 KB)

In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_loader.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_dataset = test_loader.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
callback = keras.callbacks.EarlyStopping(monitor='loss',
                                              patience=2)
history = model.fit(train_dataset,epochs=5,batch_size=100,callbacks=[callback])

Epoch 1/5


I0000 00:00:1733237002.097362    1481 service.cc:145] XLA service 0x783d08003370 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733237002.097415    1481 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  7/808 ━━━━━━━━━━━━━━━━━━━━ 15s 20ms/step - Pre: 0.7021 - Rec: 0.7021 - accuracy: 0.7021 - loss: 0.8642 

I0000 00:00:1733237011.185288    1481 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_fusion_28', 8 bytes spill stores, 8 bytes spill loads

I0000 00:00:1733237011.206943    1481 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


126/808 ━━━━━━━━━━━━━━━━━━━━ 56s 83ms/step - Pre: 0.9091 - Rec: 0.9091 - accuracy: 0.9091 - loss: 0.5398

I0000 00:00:1733237021.481430    1481 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_fusion_28', 8 bytes spill stores, 8 bytes spill loads



808/808 ━━━━━━━━━━━━━━━━━━━━ 50s 28ms/step - Pre: 0.9623 - Rec: 0.9623 - accuracy: 0.9623 - loss: 0.2156
Epoch 2/5
808/808 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - Pre: 0.9954 - Rec: 0.9954 - accuracy: 0.9954 - loss: 0.0159
Epoch 3/5
808/808 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - Pre: 0.9972 - Rec: 0.9972 - accuracy: 0.9972 - loss: 0.0094
Epoch 4/5
808/808 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - Pre: 0.9969 - Rec: 0.9969 - accuracy: 0.9969 - loss: 0.0101
Epoch 5/5
808/808 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - Pre: 0.9712 - Rec: 0.9712 - accuracy: 0.9712 - loss: 0.1615


In [11]:
model.evaluate(test_dataset)

347/347 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - Pre: 0.9503 - Rec: 0.9503 - accuracy: 0.9503 - loss: 0.1226


[0.12424933910369873,
 0.9506666660308838,
 0.9506666660308838,
 0.9506666660308838]